In [26]:
# retrieve text from PDF
from tqdm.notebook import tqdm
from PyPDF2 import PdfReader
from tqdm.notebook import tqdm
from textblob import TextBlob
import numpy as np
import spacy  
import re
from collections import Counter
from nltk.corpus import wordnet
import nltk
import pandas as pd
import numpy as np
from transformers import pipeline
import pdfplumber
import os
from difflib import SequenceMatcher
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
import pickle
from tqdm.notebook import tqdm

#### Scrapping last 50 quarterly reports

In [27]:
# Load pdf text and headings from the pickle file
pdf_texts = pickle.load(open("../Src/pdf_texts.pkl", "rb"))                        # Texts are extracted from the pdf

### Selina Model

In [28]:
nlp = spacy.load("en_core_web_sm")

def tokenize_reports(pdf_texts):
    
    # Initialize a dictionary to store the joined sentences for each report.
    joined_sentences = {}

    # Iterate over each report in pdf_texts.
    for report_name, report_text in tqdm(pdf_texts.items()): # 18 seconds

        # Split the report text into sentences.
        sentences = nlp(report_text).sents

        # Initialize a list to hold the tokenized sentences for this report.
        tokenized_report_sentences = []

        # Iterate over each sentence.
        for sentence in sentences:
            # Tokenize, lemmatize, and remove stop words and punctuation.
            tokenized = [token.lemma_ for token in sentence if not token.is_stop and not token.is_punct]
            # Add the tokenized sentence to the list.
            tokenized_report_sentences.append(tokenized)

        # Join each tokenized sentence into a single string, and store them in a list.
        joined_report_sentences = [' '.join(sentence) for sentence in tokenized_report_sentences]

        # Add the joined sentences for this report to joined_sentences.
        joined_sentences[report_name] = joined_report_sentences 

    return joined_sentences


def analyze_sentiment_of_sentences_with_keywords(joined_sentences, keywords):
    # Initialize the sentiment analysis pipeline
    sentiment_analysis = pipeline("sentiment-analysis")

    # Initialize a dictionary to store the sentences and their sentiment scores for each report
    sentiment_results_dict = {}

    # Iterate over each report
    for report_name, sentences in tqdm(joined_sentences.items()):

        # Initialize a dictionary to store the sentiment analysis results for the current report
        report_sentiment_results = {keyword: [] for keyword in keywords}

        for sentence in sentences:
            if len(sentence) < 512: 
                for keyword in keywords:
                    if keyword in sentence.lower():
                        # Analyze the sentiment of the sentence
                        sentiment_result = sentiment_analysis(sentence)[0]
                        report_sentiment_results[keyword].append(sentiment_result)

        # Add the results to the dictionary
        sentiment_results_dict[report_name] = report_sentiment_results

    return sentiment_results_dict

joined_sentences = tokenize_reports(pdf_texts)

keywords = ['revenue', 'forecast', 'profit']
sentences_with_keywords = analyze_sentiment_of_sentences_with_keywords(joined_sentences, keywords)

  0%|          | 0/50 [00:00<?, ?it/s]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


  0%|          | 0/50 [00:00<?, ?it/s]

In [29]:
# Initialize a dictionary to hold total scores for each keyword in each report
total_scores = {report: {keyword: 0 for keyword in keywords} for report in sentences_with_keywords_and_sentiment.keys()}

# Calculate total scores for each keyword in each report
for report_name, keywords_dict in sentences_with_keywords_and_sentiment.items():
    for keyword, sentiments in keywords_dict.items():
        for sentiment in sentiments:
            # If the sentiment is POSITIVE, add the score
            # If the sentiment is NEGATIVE, subtract the score
            if sentiment['label'] == 'POSITIVE':
                total_scores[report_name][keyword] += sentiment['score']
            else:
                total_scores[report_name][keyword] -= sentiment['score']

# Convert the total_scores to a DataFrame
df = pd.DataFrame(total_scores).T
df.reset_index(inplace=True)
df.columns = ['report_name', 'revenue_score', 'forecast_score', 'profit_score']
df['total_score'] = df['revenue_score'] + df['forecast_score'] + df['profit_score']

In [31]:
df.drop(['revenue_score', 'forecast_score', 'profit_score'], axis=1, inplace=True)

In [32]:
df.head()

,report_name,total_score
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,-3.337120
1,FINAL-Q1-18-Shareholder-Letter,-0.059835
2,FINAL-Q1-19-Shareholder-Letter,-2.832191
3,FINAL-Q1-20-Shareholder-Letter,-8.630516
4,FINAL-Q1-21-Shareholder-Letter,-0.299522


In [33]:
df.to_csv('../Scores/baseline_keyword_polarity.csv', index=False)

### Extract sentences with keywords

In [25]:
# def analyze_sentences_with_keywords(joined_sentences, keywords):
#     # Initialize a dictionary to store the sentences for each report
#     sentences_with_keywords_dict = {}

#     # Iterate over each report
#     for report_name, joined_sentences in tqdm(joined_sentences.items()):
#         # For each report, find sentences containing each keyword
#         sentences_with_keywords = {keyword: [] for keyword in keywords}

#         for sentence in joined_sentences:
#             for keyword in keywords:
#                 if keyword in sentence.lower():
#                     # Add the sentence to the corresponding keyword list
#                     sentences_with_keywords[keyword].append(sentence)

#         # Add the result to the dictionary
#         sentences_with_keywords_dict[report_name] = sentences_with_keywords

#     return sentences_with_keywords_dict

# # Call the function
# keywords = ['revenue', 'forecast', 'profit']
# sentences_with_keywords = analyze_sentences_with_keywords(joined_sentences, keywords)

# # Print the results
# # for report_name, keywords_sentences_dict in sentences_with_keywords.items():
# #     print(f"Report name: {report_name}")
# #     for keyword, sentences in keywords_sentences_dict.items():
# #         print(f"Keyword: {keyword}")
# #         for sentence in sentences:
# #             print(f"Sentence: {sentence}")
# #         print("\n")


  0%|          | 0/50 [00:00<?, ?it/s]

#### sentiment analysis for each sentence

In [ ]:
# def analyze_sentiment_of_sentences_with_keywords(joined_sentences, keywords):
#     # Initialize the sentiment analysis pipeline
#     sentiment_analysis = pipeline("sentiment-analysis")

#     # Initialize a dictionary to store the sentences and their sentiment scores for each report
#     sentiment_results_dict = {}

#     # Iterate over each report
#     for report_name, sentences in tqdm(joined_sentences.items()):

#         # Initialize a dictionary to store the sentiment analysis results for the current report
#         report_sentiment_results = {keyword: [] for keyword in keywords}

#         for sentence in sentences:
#             if len(sentence) < 512: 
#                 for keyword in keywords:
#                     if keyword in sentence.lower():
#                         # Analyze the sentiment of the sentence
#                         sentiment_result = sentiment_analysis(sentence)[0]
#                         report_sentiment_results[keyword].append(sentiment_result)

#         # Add the results to the dictionary
#         sentiment_results_dict[report_name] = report_sentiment_results

#     return sentiment_results_dict

# keywords = ['revenue', 'forecast', 'profit']
# sentences_with_keywords_and_sentiment = analyze_sentiment_of_sentences_with_keywords(joined_sentences, keywords) # 5 mins

#### total sentiment scores for each report

In [19]:
# Initialize a dictionary to hold total scores for each keyword in each report
total_scores = {report: {keyword: 0 for keyword in keywords} for report in sentences_with_keywords_and_sentiment.keys()}

# Calculate total scores for each keyword in each report
for report_name, keywords_dict in sentences_with_keywords_and_sentiment.items():
    for keyword, sentiments in keywords_dict.items():
        for sentiment in sentiments:
            # If the sentiment is POSITIVE, add the score
            # If the sentiment is NEGATIVE, subtract the score
            if sentiment['label'] == 'POSITIVE':
                total_scores[report_name][keyword] += sentiment['score']
            else:
                total_scores[report_name][keyword] -= sentiment['score']

# Convert the total_scores to a DataFrame
df = pd.DataFrame(total_scores).T
df.reset_index(inplace=True)
df.columns = ['report_name', 'revenue_score', 'forecast_score', 'profit_score']

In [20]:
df

,report_name,revenue_score,forecast_score,profit_score
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,-2.896049,-0.693138,0.252068
1,FINAL-Q1-18-Shareholder-Letter,0.536348,0.381404,-0.977588
2,FINAL-Q1-19-Shareholder-Letter,-0.938676,-1.893516,0.000000
3,FINAL-Q1-20-Shareholder-Letter,-4.090055,-4.030413,-0.510048
4,FINAL-Q1-21-Shareholder-Letter,0.974267,-2.263629,0.989840
5,FINAL-Q1-22-Shareholder-Letter,3.968614,-4.071353,0.041319
6,Final-Q1-23-Shareholder-Letter,-5.130473,-4.917137,-1.618782
7,FINAL-Q2-18-Shareholder-Letter,-1.327650,-5.456448,0.000000
8,FINAL-Q2-20-Shareholder-Letter-V3-with-Tables,-1.567406,-0.370292,0.000000
9,FINAL-Q2-21-Shareholder-Letter,4.227011,1.029198,2.894878


In [21]:
# Add a new column 'total_score'
df['total_score'] = df['revenue_score'] + df['forecast_score'] + df['profit_score']

# Print the results
df

,report_name,revenue_score,forecast_score,profit_score,total_score
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,-2.896049,-0.693138,0.252068,-3.337120
1,FINAL-Q1-18-Shareholder-Letter,0.536348,0.381404,-0.977588,-0.059835
2,FINAL-Q1-19-Shareholder-Letter,-0.938676,-1.893516,0.000000,-2.832191
3,FINAL-Q1-20-Shareholder-Letter,-4.090055,-4.030413,-0.510048,-8.630516
4,FINAL-Q1-21-Shareholder-Letter,0.974267,-2.263629,0.989840,-0.299522
5,FINAL-Q1-22-Shareholder-Letter,3.968614,-4.071353,0.041319,-0.061419
6,Final-Q1-23-Shareholder-Letter,-5.130473,-4.917137,-1.618782,-11.666392
7,FINAL-Q2-18-Shareholder-Letter,-1.327650,-5.456448,0.000000,-6.784098
8,FINAL-Q2-20-Shareholder-Letter-V3-with-Tables,-1.567406,-0.370292,0.000000,-1.937698
9,FINAL-Q2-21-Shareholder-Letter,4.227011,1.029198,2.894878,8.151087


In [15]:
# Initialize scaler with desired range
scaler = MinMaxScaler(feature_range=(-1, 1))

# List of columns to scale
columns_to_scale = ['profit_score', 'revenue_score', 'forecast_score', 'total_score']

# Scale each column and store in a new dataframe
scaled_sentiment = {}
for col in columns_to_scale:
    scaled_sentiment[col] = scaler.fit_transform(df[[col]]).flatten()

# Create the new dataframe with scaled columns 
scaled_sentiment['report'] = df['report_name']

scaled_sentiment = pd.DataFrame(scaled_sentiment)

scaled_sentiment



,profit_score,revenue_score,forecast_score,total_score,report
0,0.865569,0.410167,0.075900,0.508212,FINAL-Q1-11-Shareholder-Letter
1,0.132765,0.353355,0.242928,0.163406,FINAL-Q2-11-Shareholder-Letter
2,-1.000000,-0.160758,0.242537,-0.731418,FINAL-Q3-11-Shareholder-Letter
3,0.104897,-0.156242,0.421665,-0.160169,FINAL-Q4-11-Shareholder-Letter
4,0.278167,0.171772,0.063440,0.060780,FINAL-Q1-12-Shareholder-Letter
5,0.429064,0.135726,-0.106067,0.064677,FINAL-Q2-12-Shareholder-Letter
6,0.565441,0.354110,0.602222,0.458800,FINAL-Q3-12-Shareholder-Letter
7,-0.304196,-0.135041,0.064191,-0.428638,FINAL-Q4-12-Shareholder-Letter
8,-0.025810,0.336027,0.065713,0.031861,FINAL-Q1-13-Shareholder-Letter
9,0.762533,0.728592,0.422366,0.768075,FINAL-Q2-13-Shareholder-Letter


### compare sentiment scores with sentence-based manual rating

In [16]:
manual_rate_sentence = pd.read_csv("manual_check_sentence_selina.csv")
manual_rate_sentence

,report,profit,revenue,forecast,total
0,FINAL-Q1-11-Shareholder-Letter,3,1,1,5
1,FINAL-Q1-12-Shareholder-Letter,8,5,0,13
2,FINAL-Q1-13-Shareholder-Letter,2,3,1,6
3,FINAL-Q1-14-Shareholder-Letter,2,1,0,3
4,FINAL-Q1-15-Shareholder-Letter,1,0,4,5
5,FINAL-Q1-16-Shareholder-Letter,1,2,2,5
6,FINAL-Q1-17-Shareholder-Letter,0,5,1,6
7,FINAL-Q1-18-Shareholder-Letter,0,4,3,7
8,FINAL-Q1-19-Shareholder-Letter,0,4,2,6
9,FINAL-Q1-20-Shareholder-Letter,1,-4,-3,-6


In [17]:
# Initialize scaler with desired range
scaler = MinMaxScaler(feature_range=(-1, 1))

# List of columns to scale
columns_to_scale = ['profit', 'revenue', 'forecast', 'total']

# Scale each column and store in a new dataframe
scaled_manual = {}
for col in columns_to_scale:
    scaled_manual[col] = scaler.fit_transform(manual_rate_sentence[[col]]).flatten()

# Create the new dataframe with scaled columns 
scaled_manual['report'] = manual_rate_sentence['report']

scaled_manual = pd.DataFrame(scaled_manual)

scaled_manual

,profit,revenue,forecast,total,report
0,-0.384615,-0.411765,0.272727,-0.214286,FINAL-Q1-11-Shareholder-Letter
1,0.384615,0.058824,0.090909,0.357143,FINAL-Q1-12-Shareholder-Letter
2,-0.538462,-0.176471,0.272727,-0.142857,FINAL-Q1-13-Shareholder-Letter
3,-0.538462,-0.411765,0.090909,-0.357143,FINAL-Q1-14-Shareholder-Letter
4,-0.692308,-0.529412,0.818182,-0.214286,FINAL-Q1-15-Shareholder-Letter
5,-0.692308,-0.294118,0.454545,-0.214286,FINAL-Q1-16-Shareholder-Letter
6,-0.846154,0.058824,0.272727,-0.142857,FINAL-Q1-17-Shareholder-Letter
7,-0.846154,-0.058824,0.636364,-0.071429,FINAL-Q1-18-Shareholder-Letter
8,-0.846154,-0.058824,0.454545,-0.142857,FINAL-Q1-19-Shareholder-Letter
9,-0.692308,-1.000000,-0.454545,-1.000000,FINAL-Q1-20-Shareholder-Letter


In [18]:
scaled_manual = scaled_manual.rename(columns={
    'profit': 'profit_manual',
    'revenue': 'revenue_manual',
    'forecast': 'forecast_manual',
    'total': 'total_manual'
})
scaled_manual

,profit_manual,revenue_manual,forecast_manual,total_manual,report
0,-0.384615,-0.411765,0.272727,-0.214286,FINAL-Q1-11-Shareholder-Letter
1,0.384615,0.058824,0.090909,0.357143,FINAL-Q1-12-Shareholder-Letter
2,-0.538462,-0.176471,0.272727,-0.142857,FINAL-Q1-13-Shareholder-Letter
3,-0.538462,-0.411765,0.090909,-0.357143,FINAL-Q1-14-Shareholder-Letter
4,-0.692308,-0.529412,0.818182,-0.214286,FINAL-Q1-15-Shareholder-Letter
5,-0.692308,-0.294118,0.454545,-0.214286,FINAL-Q1-16-Shareholder-Letter
6,-0.846154,0.058824,0.272727,-0.142857,FINAL-Q1-17-Shareholder-Letter
7,-0.846154,-0.058824,0.636364,-0.071429,FINAL-Q1-18-Shareholder-Letter
8,-0.846154,-0.058824,0.454545,-0.142857,FINAL-Q1-19-Shareholder-Letter
9,-0.692308,-1.000000,-0.454545,-1.000000,FINAL-Q1-20-Shareholder-Letter


In [19]:
# Merge sentiment_scores_pivot and stock_price DataFrames
scaled_df = pd.merge(scaled_sentiment, scaled_manual, on='report', how='left')

# Print the results
scaled_df


,profit_score,revenue_score,forecast_score,total_score,report,profit_manual,revenue_manual,forecast_manual,total_manual
0,0.865569,0.410167,0.075900,0.508212,FINAL-Q1-11-Shareholder-Letter,-0.384615,-0.411765,0.272727,-0.214286
1,0.132765,0.353355,0.242928,0.163406,FINAL-Q2-11-Shareholder-Letter,-0.692308,-0.529412,-0.090909,-0.571429
2,-1.000000,-0.160758,0.242537,-0.731418,FINAL-Q3-11-Shareholder-Letter,-1.000000,-0.411765,0.272727,-0.500000
3,0.104897,-0.156242,0.421665,-0.160169,FINAL-Q4-11-Shareholder-Letter,-0.230769,-0.294118,0.454545,0.000000
4,0.278167,0.171772,0.063440,0.060780,FINAL-Q1-12-Shareholder-Letter,0.384615,0.058824,0.090909,0.357143
5,0.429064,0.135726,-0.106067,0.064677,FINAL-Q2-12-Shareholder-Letter,1.000000,-0.058824,-0.272727,0.428571
6,0.565441,0.354110,0.602222,0.458800,FINAL-Q3-12-Shareholder-Letter,-0.230769,-0.529412,0.272727,-0.214286
7,-0.304196,-0.135041,0.064191,-0.428638,FINAL-Q4-12-Shareholder-Letter,-0.076923,-0.058824,0.454545,0.214286
8,-0.025810,0.336027,0.065713,0.031861,FINAL-Q1-13-Shareholder-Letter,-0.538462,-0.176471,0.272727,-0.142857
9,0.762533,0.728592,0.422366,0.768075,FINAL-Q2-13-Shareholder-Letter,-0.692308,-0.411765,0.090909,-0.428571


In [30]:
scaled_df.to_csv('sentiment_score.csv',index=False)

In [25]:
# Calculate the mean of the actual values
manual = scaled_df["total_manual"]
sentiment = scaled_df["total_score"]
# calculate MAE and MSE
mae = mean_absolute_error(manual, sentiment)
mse = mean_squared_error(manual, sentiment)

print("MAE:", mae)
print("MSE:", mse)

MAE: 0.7257173870788294
MSE: 0.642762387542966


In [25]:
x = scaled_df[['total_score']]
y = scaled_df['total_manual']

# x = sm.add_constant(x)

model = sm.OLS(y, x).fit()
predictions = model.predict(x) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:           total_manual   R-squared (uncentered):                   0.105
Model:                            OLS   Adj. R-squared (uncentered):              0.087
Method:                 Least Squares   F-statistic:                              5.759
Date:                Thu, 24 Aug 2023   Prob (F-statistic):                      0.0203
Time:                        18:44:51   Log-Likelihood:                         -21.028
No. Observations:                  50   AIC:                                      44.06
Df Residuals:                      49   BIC:                                      45.97
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

#### compare sentiment scores with full text average rating

In [31]:
full_text_manual_ratings = pd.read_csv("average_rating.csv")
full_text_manual_ratings

,report,average_rating
0,FINAL-Q3-16-Shareholder-Letter,0.483333
1,FINAL-Q1-11-Shareholder-Letter,0.433333
2,FINAL-Q1-12-Shareholder-Letter,0.000000
3,FINAL-Q1-13-Shareholder-Letter,0.000000
4,FINAL-Q1-14-Shareholder-Letter,0.333333
5,FINAL-Q1-14-Shareholder-Letter,0.166667
6,FINAL-Q1-15-Shareholder-Letter,0.333333
7,FINAL-Q1-16-Shareholder-Letter,0.500000
8,FINAL-Q1-17-Shareholder-Letter,0.433333
9,FINAL-Q1-18-Shareholder-Letter,0.500000


In [32]:
# Initialize scaler with desired range
scaler = MinMaxScaler(feature_range=(-1, 1))

# Scale each column and store in a new dataframe
scaled_average_rating = {}
scaled_average_rating['scaled_full_text_manual_ratings'] = scaler.fit_transform(full_text_manual_ratings[['average_rating']]).flatten()

# Create the new dataframe with scaled columns 
scaled_average_rating['report'] = full_text_manual_ratings['report']
scaled_average_rating = pd.DataFrame(scaled_average_rating)

scaled_average_rating

,scaled_full_text_manual_ratings,report
0,0.658537,FINAL-Q3-16-Shareholder-Letter
1,0.512195,FINAL-Q1-11-Shareholder-Letter
2,-0.756098,FINAL-Q1-12-Shareholder-Letter
3,-0.756098,FINAL-Q1-13-Shareholder-Letter
4,0.219512,FINAL-Q1-14-Shareholder-Letter
5,-0.268293,FINAL-Q1-14-Shareholder-Letter
6,0.219512,FINAL-Q1-15-Shareholder-Letter
7,0.707317,FINAL-Q1-16-Shareholder-Letter
8,0.512195,FINAL-Q1-17-Shareholder-Letter
9,0.707317,FINAL-Q1-18-Shareholder-Letter


In [33]:
# Merge
scaled_full_df = pd.merge(scaled_average_rating, scaled_sentiment, left_on='report', right_on='report')
# Print the results
scaled_full_df

,scaled_full_text_manual_ratings,report,profit_score,revenue_score,forecast_score,total_score
0,0.658537,FINAL-Q3-16-Shareholder-Letter,0.954117,0.640087,0.476666,0.810722
1,0.512195,FINAL-Q1-11-Shareholder-Letter,0.865569,0.410167,0.075900,0.508212
2,-0.756098,FINAL-Q1-12-Shareholder-Letter,0.278167,0.171772,0.063440,0.060780
3,-0.756098,FINAL-Q1-13-Shareholder-Letter,-0.025810,0.336027,0.065713,0.031861
4,0.219512,FINAL-Q1-14-Shareholder-Letter,0.316793,0.491190,0.361893,0.376400
5,-0.268293,FINAL-Q1-14-Shareholder-Letter,0.316793,0.491190,0.361893,0.376400
6,0.219512,FINAL-Q1-15-Shareholder-Letter,0.741008,0.479480,0.486602,0.600362
7,0.707317,FINAL-Q1-16-Shareholder-Letter,0.409733,0.608415,0.008508,0.413622
8,0.512195,FINAL-Q1-17-Shareholder-Letter,0.862506,0.801051,0.957871,1.000000
9,0.707317,FINAL-Q1-18-Shareholder-Letter,0.673439,0.705988,0.497839,0.729027


In [36]:
# Calculate the mean of the actual values
manual = scaled_full_df['scaled_full_text_manual_ratings']
sentiment = scaled_full_df[['total_score']]
# calculate MAE and MSE
mae = mean_absolute_error(manual, sentiment)
mse = mean_squared_error(manual, sentiment)

print("MAE:", mae)
print("MSE:", mse)

MAE: 0.48064281774896
MSE: 0.4070166032397313


In [34]:
x = scaled_full_df[['total_score']]
y = scaled_full_df['scaled_full_text_manual_ratings']

# x = sm.add_constant(x)

model = sm.OLS(y, x).fit()
predictions = model.predict(x) 

print_model = model.summary()
print(print_model)

                                       OLS Regression Results                                       
Dep. Variable:     scaled_full_text_manual_ratings   R-squared (uncentered):                   0.160
Model:                                         OLS   Adj. R-squared (uncentered):              0.143
Method:                              Least Squares   F-statistic:                              9.350
Date:                             Fri, 25 Aug 2023   Prob (F-statistic):                     0.00361
Time:                                     21:21:49   Log-Likelihood:                         -38.112
No. Observations:                               50   AIC:                                      78.22
Df Residuals:                                   49   BIC:                                      80.14
Df Model:                                        1                                                  
Covariance Type:                         nonrobust                                         

### Stock price processing

In [ ]:
stock_price = pd.read_csv("NFLX.csv")
stock_price.loc[:, "open_close_diff"] = stock_price.Close - stock_price.Open
stock_price

,Quater,Date,Open,High,Low,Close,Adj Close,Volume,open_close_diff
0,FINAL-Q2-23-Shareholder-Letter,19/07/2023,476.860000,485.000000,470.000000,477.590000,477.590000,20210900,0.730000
1,FINAL-Q1-23-Shareholder-Letter,18/04/2023,335.000000,337.190002,330.500000,333.700012,333.700012,17944500,-1.299988
2,FINAL-Q4-22-Shareholder-Letter,19/01/2023,322.570007,324.890015,313.390015,315.779999,315.779999,18008200,-6.790008
3,FINAL-Q3-22-Shareholder-Letter,18/10/2022,249.800003,250.369995,237.729996,240.860001,240.860001,25776700,-8.940002
4,FINAL-Q2-22-Shareholder-Letter,19/07/2022,193.020004,201.970001,188.399994,201.630005,201.630005,28178700,8.610001
5,FINAL-Q1-22-Shareholder-Letter,19/04/2022,333.220001,351.679993,333.220001,348.609985,348.609985,20906900,15.389984
6,FINAL-Q4-21-Shareholder-Letter,20/01/2022,517.750000,526.640015,506.929993,508.250000,508.250000,12659000,-9.500000
7,FINAL-Q3-21-Shareholder-Letter,19/10/2021,636.969971,641.000000,632.299988,639.000000,639.000000,7633100,2.030029
8,FINAL-Q2-21-Shareholder-Letter,20/07/2021,526.070000,536.640000,520.300000,531.050000,531.050000,"6,930,400",4.980000
9,FINAL-Q1-21-Shareholder-Letter,20/04/2021,554.420000,563.560000,546.300000,549.570000,549.570000,"11,257,600",-4.850000


### match the polarity score with open_close_diff 

In [ ]:
# Merge sentiment_scores_pivot and stock_price DataFrames
final_df = pd.merge(df, stock_price, left_on='report_name', right_on='Quater')
final_df = final_df.drop(columns=['Quater'])
# Print the results
final_df

,report_name,revenue_score,forecast_score,profit_score,total_score,Date,Open,High,Low,Close,Adj Close,Volume,open_close_diff
0,FINAL-Q1-16-Shareholder-Letter,0.005883,-2.298231,-2.769693,-5.062041,18/04/2016,109.900002,110.699997,106.019997,108.400002,108.400002,32816400,-1.500000
1,FINAL-Q2-16-Shareholder-Letter,1.138111,-2.341845,-1.684838,-2.888572,18/07/2016,98.430000,99.839996,97.239998,98.809998,98.809998,28669700,0.379998
2,FINAL-Q3-16-Shareholder-Letter,0.495398,0.301537,2.947956,3.744891,17/10/2016,100.500000,100.730003,98.379997,99.800003,99.800003,26589500,-0.699997
3,FINAL-Q4-16-Shareholder-Letter,2.965716,-0.221106,1.975570,4.720181,18/01/2017,133.210007,133.649994,131.059998,133.259995,133.259995,16168600,0.049988
4,FINAL-Q1-17-Shareholder-Letter,2.983195,2.973760,1.985773,7.942729,17/04/2017,144.429993,147.320007,144.429993,147.250000,147.250000,16364700,2.820007
5,FINAL-Q2-17-Shareholder-Letter,1.336397,-1.088799,3.131801,3.379399,17/07/2017,162.910004,163.550003,160.020004,161.699997,161.699997,17496700,-1.210007
6,FINAL-Q3-17-Shareholder-Letter,1.987605,1.504858,0.525840,4.018302,16/10/2017,201.949997,202.830002,197.860001,202.679993,202.679993,22105400,0.729996
7,FINAL-Q4-17-Shareholder-Letter,-2.896049,-1.853489,0.204556,-4.544982,22/01/2018,222.000000,227.789993,221.199997,227.580002,227.580002,17703300,5.580002
8,FINAL-Q1-18-Shareholder-Letter,1.513936,0.419115,0.000000,1.933051,16/04/2018,315.989990,316.100006,304.000000,307.779999,307.779999,20307900,-8.209991
9,FINAL-Q2-18-Shareholder-Letter,-1.327654,-5.456454,0.000000,-6.784108,16/07/2018,398.980011,403.359985,391.750000,400.480011,400.480011,22960000,1.500000


In [ ]:
final_df['match'] = np.sign(final_df['total_score']) == np.sign(final_df['open_close_diff'])

# Print the results
final_df

,report_name,revenue_score,forecast_score,profit_score,total_score,Date,Open,High,Low,Close,Adj Close,Volume,open_close_diff,match
0,FINAL-Q1-16-Shareholder-Letter,0.005883,-2.298231,-2.769693,-5.062041,18/04/2016,109.900002,110.699997,106.019997,108.400002,108.400002,32816400,-1.500000,True
1,FINAL-Q2-16-Shareholder-Letter,1.138111,-2.341845,-1.684838,-2.888572,18/07/2016,98.430000,99.839996,97.239998,98.809998,98.809998,28669700,0.379998,False
2,FINAL-Q3-16-Shareholder-Letter,0.495398,0.301537,2.947956,3.744891,17/10/2016,100.500000,100.730003,98.379997,99.800003,99.800003,26589500,-0.699997,False
3,FINAL-Q4-16-Shareholder-Letter,2.965716,-0.221106,1.975570,4.720181,18/01/2017,133.210007,133.649994,131.059998,133.259995,133.259995,16168600,0.049988,True
4,FINAL-Q1-17-Shareholder-Letter,2.983195,2.973760,1.985773,7.942729,17/04/2017,144.429993,147.320007,144.429993,147.250000,147.250000,16364700,2.820007,True
5,FINAL-Q2-17-Shareholder-Letter,1.336397,-1.088799,3.131801,3.379399,17/07/2017,162.910004,163.550003,160.020004,161.699997,161.699997,17496700,-1.210007,False
6,FINAL-Q3-17-Shareholder-Letter,1.987605,1.504858,0.525840,4.018302,16/10/2017,201.949997,202.830002,197.860001,202.679993,202.679993,22105400,0.729996,True
7,FINAL-Q4-17-Shareholder-Letter,-2.896049,-1.853489,0.204556,-4.544982,22/01/2018,222.000000,227.789993,221.199997,227.580002,227.580002,17703300,5.580002,False
8,FINAL-Q1-18-Shareholder-Letter,1.513936,0.419115,0.000000,1.933051,16/04/2018,315.989990,316.100006,304.000000,307.779999,307.779999,20307900,-8.209991,False
9,FINAL-Q2-18-Shareholder-Letter,-1.327654,-5.456454,0.000000,-6.784108,16/07/2018,398.980011,403.359985,391.750000,400.480011,400.480011,22960000,1.500000,False


In [ ]:
counts = final_df['match'].value_counts()
counts

True     16
False    14
Name: match, dtype: int64

In [ ]:
count_true = final_df['match'].sum()
accuracy= count_true/30
accuracy

0.5333333333333333